In [1]:
import pandas as pd
from PIL import Image
from datetime import datetime, timedelta
import requests
from data_mining.get_sat2_images import call_sentinel
import data_mining.read_weather as weather
from tifffile import imwrite, imread
import numpy as np
import time
from openmeteo_requests.Client import OpenMeteoRequestsError
import shutil
import os

# gather fire data

In [ ]:
import fire_news.read_firenews as firenews
import fire_confidence.fetch_fire_confidence as fire_confidence
import fire_labels.fire_labels as fire_labels
firenews.run()
fire_confidence.run()
fire_labels.run()

## or read from file

In [2]:
# fire_data= pd.read_csv("fire_news.csv")
fire_data = pd.read_csv("fire_labels/fire_labels_final.csv")
fire_data = fire_data[["latitude", "longitude", "date"]][::-1]
# fire_data.rename(columns={'latitude_clean':'latitude','longitude_clean':'longitude'},inplace=True)
fire_data.drop_duplicates(inplace=True)

# get images from sentinel 2

In [3]:
# Your client credentials
with open('../config') as f:
    contents = f.readlines()[0].split(" ")
    client_id = contents[0]
    client_secret = contents[1]

In [4]:
main_folder="sentinel_images"
for index, row in fire_data.iterrows():
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    # yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
    # time_interval=(yesterday, day)
    # call_sentinel(client_id,client_secret,row,time_interval,save=True)
    dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-9, 0)]
    for date in dates:
        day_date = datetime.strptime(date, "%Y-%m-%d")
        yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
        time_interval=(yesterday, date)
        call_sentinel(client_id,client_secret,row,time_interval,save=True,folder=main_folder)
            

## resize images to be uniform

In [5]:
def resize_image(input_data, target_size):
    # input_data should be a numpy array with shape (height, width, 13)
    height, width, channels = input_data.shape
    # Create a resized array
    resized_data = np.zeros((*target_size, channels), dtype=input_data.dtype)

    # Compute scaling factors
    scale_height = target_size[0] / height
    scale_width = target_size[1] / width

    # Iterate through each pixel and scale the values
    for h in range(target_size[0]):
        for w in range(target_size[1]):
            h_orig = int(h / scale_height)
            w_orig = int(w / scale_width)
            resized_data[h, w, :] = input_data[h_orig, w_orig, :]

    return resized_data

# Specify the directory path
directory_path = 'sentinel_images'  # Replace with your directory path
# Get a list of all entries (files and subdirectories) in the specified directory
entries = os.listdir(directory_path)
# Iterate over each entry
rows=[]
for entry in entries:
    lat,long=entry.split(",")
    # Create the full path to the entry
    full_path = os.path.join(directory_path, entry)

    # Check if the entry is a directory
    if os.path.isdir(full_path):
        print(f"Subfolder: {entry}")

        # List all files and subfolders within the subfolder
        dates = os.listdir(full_path)
        for date in dates:
            rows.append([lat,long,date.split(".")[0]])
            print(f"  - {date}")
            file = f"sentinel_images/{entry}/{date}"
            image = imread(file)
            resized_image=resize_image(image,(190,190))
            imwrite(file, resized_image)
print("done resizing images")

# get weather data

## only for dates where we found picture

In [4]:
# Specify the directory path
directory_path = 'sentinel_images'  # Replace with your directory path
# Get a list of all entries (files and subdirectories) in the specified directory
entries = os.listdir(directory_path)
# Iterate over each entry
rows=[]
for entry in entries:
    lat,long=entry.split(",")
    # Create the full path to the entry
    full_path = os.path.join(directory_path, entry)

    # Check if the entry is a directory
    if os.path.isdir(full_path):
        print(f"Subfolder: {entry}")

        # List all files and subfolders within the subfolder
        dates = os.listdir(full_path)
        for date in dates:
            rows.append([lat,long,date.split(".")[0]])
            print(f"  - {date}")
    else:
        print(f"File: {entry}")

fire_weather = pd.DataFrame(rows, columns=["latitude","longitude","date"])

In [5]:
openmeteo_session=weather.start_openmeteo_session()
def try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=False):
    try:
        weather.get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)
    except OpenMeteoRequestsError as e:
        print(e)
        time.sleep(61)
        try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)
for index, row in fire_weather.iterrows():
    print(index)    
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    week_ago = (day_date - timedelta(days=9)).strftime("%Y-%m-%d")
    time_interval=(week_ago, day)
    try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)

# create negative samples

In [ ]:
import negative_sampling
negative_sampling.main(n=500)

# clean datasets

In [ ]:
import remove_small_imagesets
remove_small_imagesets.remove_folders_with_few_files()
# move all images to one folder
source_dir = 'sentinel_images_no_fire'
target_dir = 'sentinel_images'
file_names = os.listdir(source_dir)
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

# train and save the model

In [ ]:
from training.model import train_model
train_model("../") # save to root folder